In [142]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm

import pytorch_lightning as pl

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

1.8.1
False


In [86]:
class VAEResBlock(torch.nn.Module):
    def __init__(self, in_channels, kernel_size=5) -> None:
        super().__init__()

        self.activation = F.relu

        self.bn1 = torch.nn.BatchNorm2d(in_channels)

        self.bn2 = torch.nn.BatchNorm2d(in_channels * 2)

        self.conv1 = torch.nn.Conv2d(
            in_channels,
            in_channels * 2,
            kernel_size=5,
            stride=2,
            padding=2
        )


        self.conv2 = torch.nn.Conv2d(
            in_channels * 2,
            in_channels * 2,
            kernel_size=5,
            stride=1,
            padding=2,
        )

        self.conv_skip = torch.nn.Conv2d(
            in_channels,
            in_channels * 2,
            kernel_size=5,
            stride=2,
            padding=2

        )

    def forward(self, x):
        print("input: ", x.shape)
        out = self.conv1(self.activation(self.bn1(x)))
        print("conv1: ", out.shape)
        out = self.conv2(self.activation(self.bn2(out)))
        print("conv2: ", out.shape)
        out += self.conv_skip(x)
        print("+skip: ", out.shape)
        #out = self.pool(out)
        #print(out.shape)
        return out

In [136]:
class Reverse_VAEResBlock(torch.nn.Module):
    def __init__(self, in_channels, kernel_size=5) -> None:
        super().__init__()

        self.activation = F.relu

        self.bn1 = torch.nn.BatchNorm2d(in_channels)

        self.bn2 = torch.nn.BatchNorm2d(in_channels // 2)

        self.deconv1 = torch.nn.ConvTranspose2d(
            in_channels,
            in_channels // 2,
            kernel_size=5,
            stride=2,
            padding=2,
            output_padding = 1
        )


        self.deconv2 = torch.nn.ConvTranspose2d(
            in_channels // 2,
            in_channels // 2,
            kernel_size=5,
            stride=1,
            padding=2,
        )

        self.deconv_skip = torch.nn.ConvTranspose2d(
            in_channels,
            in_channels // 2,
            kernel_size=5,
            stride=2,
            padding=2,
            output_padding = 1
        )

    def forward(self, x):
        print("recode: ", x.shape)
        out = self.deconv1(self.activation(self.bn1(x)))
        print("deconv1: ", out.shape)
        out = self.deconv2(self.activation(self.bn2(out)))
        print("deconv2: ", out.shape)
        out += self.deconv_skip(x)
        print("+skip: ", out.shape)
        #out = self.pool(out)
        #print(out.shape)
        return out

In [93]:
boop = nn.ConvTranspose2d(16, 8, kernel_size= 5, stride = 2, padding=2)

torch.Size([16, 8, 8])

In [141]:
img_shape = (4, 32, 32)

tst_img = torch.randn(1, *img_shape)

print("Flat size: ", torch.numel(tst_img))

tst_res = VAEResBlock(img_shape[0])

new = tst_res(tst_img)

tst_res2 = Reverse_VAEResBlock(new.shape[1])

#tst_res2 = nn.ConvTranspose2d(8, 4, kernel_size=5, stride = 2, padding = 2, output_padding = 1)

#tst_res2 = nn.ConvTranspose2d(8, 4, kernel_size=3, stride = 2)

#tst_res2 = VAEResBlock(new.shape[1])

print("---\nPassing to decoder\n---")

new = tst_res2(new)
#print("deconv: ", new.shape)

#tst_res3 = nn.ConvTranspose2d(4, 4, kernel_size=5, stride = 1, padding = 2)

#new = tst_res3(new)
#print("deconv2: ", new.shape)

print("Flat size: ", torch.numel(new))
print("---")
print("Down by factor of: ", torch.numel(tst_img)/torch.numel(new))



Flat size:  4096
input:  torch.Size([1, 4, 32, 32])
conv1:  torch.Size([1, 8, 16, 16])
conv2:  torch.Size([1, 8, 16, 16])
+skip:  torch.Size([1, 8, 16, 16])
---
Passing to decoder
---
recode:  torch.Size([1, 8, 16, 16])
deconv1:  torch.Size([1, 4, 32, 32])
deconv2:  torch.Size([1, 4, 32, 32])
+skip:  torch.Size([1, 4, 32, 32])
Flat size:  4096
---
Down by factor of:  1.0


In [45]:
np.log2(4096/128)

5.0

In [47]:
(256*256)/(2**5)

2048.0

In [46]:
1*256*256


65536

16

In [153]:
x = np.arange(1,10)
y = (x - 1) // 2

print(x)
print(y)

[1 2 3 4 5 6 7 8 9]
[0 0 1 1 2 2 3 3 4]


In [156]:

(lambda x : (x % 2))(x)

array([1, 0, 1, 0, 1, 0, 1, 0, 1])

In [162]:
x = 4
x/=2
x

2.0

In [187]:
from Lit_Ab_VAE import Ab_VAE

In [190]:
img_shape = (3,32,32)
model = Ab_VAE(img_shape)

NameError: name 'res_channel' is not defined